In [19]:
import pandas as pd
import requests
import re
import json
from bs4 import BeautifulSoup

In [20]:
url = 'https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410020101&pickMembers%5B0%5D=1.3&pickMembers%5B1%5D=2.1&cubeTimeFrame.startMonth=06&cubeTimeFrame.startYear=2022&cubeTimeFrame.endMonth=10&cubeTimeFrame.endYear=2022&referencePeriods=20220601%2C20221001'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [64]:
date = soup.find_all('meta', attrs={'name': 'dcterms.issued'})[0]['content']


2022-12-22


In [21]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [22]:
result = find_between(soup.prettify(), 'tableContainerElement = $(".tableContainer").clone();', 'window.addEventListener("resize", function() {') + 'end'
data = find_between(result, 'prepareTable(', '\n')[:-2]
json_data = json.loads(data)

In [23]:
rows = json_data['rows']

new_rows = []
for row in rows:
    values = row['values']
    for value in values:
        new_rows.append(value["value"])


In [43]:
category = next(item for item in json_data['headers']["columnHeaders"] if item["name"] == "Geography")
category_selected = category["values"][0]["value"]


In [24]:
headers = next(item for item in json_data['headers']["columnHeaders"] if item["name"] == "Reference period")
header_values = []
for item in headers["values"]:
        header_values.append(item["value"])

In [25]:
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

In [26]:
rows_values = {}
key = ""
data = []
index = 0
for row in new_rows:
    temp_data = {}
    if not isfloat(row):
        key = row
        rows_values[key] = []
        data = []
        index = 0
    if isfloat(row):
        data.append(float(row))
        rows_values[key] = data
        index += 1
print(rows_values)


{'Industrial aggregate including unclassified businesses': [80279.0, 78991.0, 77562.0, 79018.0, 76721.0], 'Industrial aggregate excluding unclassified businesses': [78655.0, 77352.0, 75944.0, 77479.0, 75288.0], 'Goods producing industries': [11857.0, 12097.0, 12410.0, 11951.0, 11786.0], 'Forestry, logging and support': [0.0, 0.0, 0.0, 0.0, 0.0], 'Mining, quarrying, and oil and gas extraction': [0.0, 0.0, 0.0, 0.0, 0.0], 'Utilities': [0.0, 0.0, 0.0, 0.0, 0.0], 'Construction': [4801.0, 5103.0, 5354.0, 5081.0, 5025.0], 'Manufacturing': [6523.0, 6449.0, 6517.0, 6320.0, 6268.0], 'Service producing industries': [66798.0, 65255.0, 63533.0, 65528.0, 63502.0], 'Trade': [14198.0, 13665.0, 12919.0, 12818.0, 12417.0], 'Transportation and warehousing': [3031.0, 3088.0, 3186.0, 3119.0, 3100.0], 'Information and cultural industries': [0.0, 0.0, 0.0, 0.0, 0.0], 'Finance and insurance': [1934.0, 1919.0, 1928.0, 1914.0, 1964.0], 'Real estate and rental and leasing': [913.0, 949.0, 953.0, 915.0, 925.0], 

In [34]:
test_data = []
keys = []
for key, value in rows_values.items():
    index=0
    temp_data = {}
    temp_data["key"] = key
    key = key.replace(" ", "_").replace(",","").replace("(","").replace(")","").replace("-","_").replace("__","_").lower()
    keys.append(key)
    for i in value:
        temp_data[header_values[index]] = i
        index += 1
    test_data.append(temp_data)

In [65]:
df = pd.DataFrame(test_data).transpose().drop("key")
df.columns = keys
df["category"] = category_selected
df["date"] = soup.find_all('meta', attrs={'name': 'dcterms.issued'})[0]['content']
